In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

dlp_jan_2025_nppe_3_path = kagglehub.competition_download('dlp-jan-2025-nppe-3')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --quiet timm


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
from tqdm import tqdm
from math import log10
import glob

In [ ]:
# ⚙️ Config
IMG_SIZE = (128, 80)
HR_SIZE = (512, 320)  # Downscaled from 1024x640 to reduce memory
BATCH_SIZE = 2
EPOCHS = 30
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 📁 Paths
TRAIN_LOW = "/kaggle/input/dlp-jan-2025-nppe-3/archive/train/train"
TRAIN_HIGH = "/kaggle/input/dlp-jan-2025-nppe-3/archive/train/gt"
VAL_LOW = "/kaggle/input/dlp-jan-2025-nppe-3/archive/val/val"
VAL_HIGH = "/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt"

In [ ]:
# 🔁 Transform
transform_lr = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor()
])

transform_hr = transforms.Compose([
    transforms.Resize(HR_SIZE),
    transforms.ToTensor()
])

In [ ]:
# 📦 Dataset
class SuperResolutionDataset(Dataset):
    def __init__(self, low_dir, high_dir, transform_lr, transform_hr):
        self.low_paths = sorted(glob.glob(f"{low_dir}/*.png"))
        self.high_paths = sorted(glob.glob(f"{high_dir}/*.png"))
        self.transform_lr = transform_lr
        self.transform_hr = transform_hr

    def __len__(self):
        return len(self.low_paths)

    def __getitem__(self, idx):
        low = Image.open(self.low_paths[idx]).convert("RGB")
        high = Image.open(self.high_paths[idx]).convert("RGB")
        return self.transform_lr(low), self.transform_hr(high)

In [ ]:
# 📊 PSNR
def calculate_psnr(pred, target):
    mse = F.mse_loss(pred, target)
    return 10 * log10(1 / mse.item())

# 🧠 Model: Efficient Restormer Lite
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(dim, hidden_dim, 1),
            nn.GELU(),
            nn.Conv2d(hidden_dim, dim, 1)
        )

    def forward(self, x):
        return self.net(x)

class LKA(nn.Module):  # Large Kernel Attention
    def __init__(self, dim):
        super().__init__()
        self.conv1 = nn.Conv2d(dim, dim, kernel_size=5, padding=2, groups=dim)
        self.conv2 = nn.Conv2d(dim, dim, kernel_size=7, padding=9, dilation=3, groups=dim)
        self.conv3 = nn.Conv2d(dim, dim, kernel_size=1)

    def forward(self, x):
        u = x
        attn = self.conv1(x)
        attn = self.conv2(attn)
        attn = self.conv3(attn)
        return u * attn

class TransformerBlockLite(nn.Module):
    def __init__(self, dim, ff_dim):
        super().__init__()
        self.norm1 = nn.BatchNorm2d(dim)
        self.attn = LKA(dim)
        self.norm2 = nn.BatchNorm2d(dim)
        self.ffn = FeedForward(dim, ff_dim)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.ffn(self.norm2(x))
        return x

class RestormerLite(nn.Module):
    def __init__(self, in_channels=3, dim=32, num_blocks=3, ff_dim=64):
        super().__init__()
        self.shallow_feat = nn.Conv2d(in_channels, dim, 3, 1, 1)
        self.encoder = nn.Sequential(*[TransformerBlockLite(dim, ff_dim) for _ in range(num_blocks)])
        self.upsample = nn.Sequential(
            nn.Conv2d(dim, dim * 4, 3, 1, 1),
            nn.PixelShuffle(2),
            nn.Conv2d(dim, dim * 4, 3, 1, 1),
            nn.PixelShuffle(2)
        )
        self.output = nn.Conv2d(dim, 3, 3, 1, 1)

    def forward(self, x):
        x = self.shallow_feat(x)
        x = self.encoder(x)
        x = self.upsample(x)
        return self.output(x)

# 📶 Training Function
def train_model(model, train_loader, val_loader, device, epochs=15):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
    criterion = nn.L1Loss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for low, high in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            low, high = low.to(device), high.to(device)
            output = model(low)
            loss = criterion(output, high)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} - Train Loss: {avg_loss:.4f}")

        # 🔍 Validation
        model.eval()
        psnrs = []
        with torch.no_grad():
            for low, high in val_loader:
                low, high = low.to(device), high.to(device)
                output = model(low)
                psnrs.append(calculate_psnr(output, high))
        print(f"Validation PSNR: {np.mean(psnrs):.2f} dB")

In [ ]:
# 🚀 Prepare + Train
train_dataset = SuperResolutionDataset(TRAIN_LOW, TRAIN_HIGH, transform_lr, transform_hr)
val_dataset = SuperResolutionDataset(VAL_LOW, VAL_HIGH, transform_lr, transform_hr)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=2)

model = RestormerLite()
train_model(model, train_loader, val_loader, DEVICE, epochs=EPOCHS)

Epoch 1/30: 100%|██████████| 553/553 [00:18<00:00, 30.48it/s]

Epoch 1 - Train Loss: 0.0277


Validation PSNR: 35.40 dB


Epoch 2/30: 100%|██████████| 553/553 [00:17<00:00, 31.14it/s]

Epoch 2 - Train Loss: 0.0154


...

Epoch 29 - Train Loss: 0.0110


Validation PSNR: 37.92 dB


Epoch 30/30: 100%|██████████| 553/553 [00:17<00:00, 31.19it/s]

Epoch 30 - Train Loss: 0.0109


Validation PSNR: 37.87 dB


In [ ]:
import shutil

# 📁 Test image directory and save path
TEST_DIR = "/kaggle/input/dlp-jan-2025-nppe-3/archive/test"
SAVE_DIR = "/kaggle/working/test_preds"
os.makedirs(SAVE_DIR, exist_ok=True)

# 🧪 Load and predict on test set
test_paths = sorted(glob.glob(f"{TEST_DIR}/*.png"))
model.eval()

with torch.no_grad():
    for path in tqdm(test_paths, desc="Running Test Inference"):
        img = Image.open(path).convert("RGB")
        img_tensor = transform_lr(img).unsqueeze(0).to(DEVICE)

        with torch.cuda.amp.autocast():  # optional for speed
            pred = model(img_tensor)

        pred_img = pred.squeeze().cpu().clamp(0, 1).permute(1, 2, 0).numpy()
        pred_img = (pred_img * 255).astype(np.uint8)
        pred_pil = Image.fromarray(pred_img)

        fname = os.path.basename(path)
        pred_pil.save(os.path.join(SAVE_DIR, fname))


Running Test Inference:   0%|          | 0/60 [00:00<?, ?it/s]<ipython-input-42-2dafb6ea00e6>:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # optional for speed
Running Test Inference: 100%|██████████| 60/60 [00:04<00:00, 12.57it/s]


In [ ]:
import os
import shutil

# Define actual prediction folder and dummy folder expected by submission.py
actual_preds = '/kaggle/working/test_preds'
dummy_folder = '/kaggle/working/path_to_images'

# Remove the dummy folder if it exists (fresh start)
if os.path.exists(dummy_folder):
    shutil.rmtree(dummy_folder)

# Create dummy folder
os.makedirs(dummy_folder, exist_ok=True)

# Copy all predicted images to the dummy folder
for file_name in os.listdir(actual_preds):
    if file_name.endswith('.png'):
        shutil.copy(os.path.join(actual_preds, file_name), os.path.join(dummy_folder, file_name))

# Move into the working directory where submission.py expects to write 'submission.csv'
os.chdir('/kaggle/working')

# Now run the uneditable script
!python /kaggle/input/dlp-jan-2025-nppe-3/submission.py


Successfully saved to submission.csv
